<a href="https://colab.research.google.com/github/ntavakoulnia/ntavakoulnia/blob/main/Copy_of_Student_MLE_MiniProject_Recommendation_Engines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project: Recommendation Engines

Recommendation engines are algorithms designed to provide personalized suggestions or recommendations to users. These systems analyze user behavior, preferences, and interactions with items (products, movies, music, articles, etc.) to predict and offer items that users are likely to be interested in. Recommendation engines play a crucial role in enhancing user experience, driving engagement, and increasing conversion rates in various applications, including e-commerce, entertainment, content platforms, and more.

There are generally two approaches taken in collaborative filtering and content-based recommendation engines:

**1. Collaborative Filtering:**
Collaborative Filtering is a popular approach to building recommendation systems that leverages the collective behavior of users to make personalized recommendations. It is based on the idea that users who have agreed in the past will likely agree in the future. There are two main types of collaborative filtering:

- **User-based Collaborative Filtering:** This method finds users similar to the target user based on their past interactions (e.g., ratings or purchases). It then recommends items that similar users have liked but the target user has not interacted with yet.

- **Item-based Collaborative Filtering:** In this approach, the system identifies similar items based on user interactions. It recommends items that are similar to the ones the target user has already liked or interacted with.

Collaborative filtering does not require any explicit information about items but relies on the similarity between users or items. It is effective in capturing complex patterns and can provide serendipitous recommendations. However, it suffers from the cold-start problem (i.e., difficulty in recommending to new users or items with no interactions) and scalability challenges in large datasets.

**2. Content-Based Recommendation:**
Content-based recommendation is an alternative approach to building recommendation systems that focuses on the attributes or features of items and users. It leverages the characteristics of items to make recommendations. The key steps involved in content-based recommendation are:

- **Feature Extraction:** For each item, relevant features are extracted. For movies, these features could be genre, director, actors, and plot summary.

- **User Profile:** A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.

- **Similarity Calculation:** The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity or Euclidean distance.

- **Recommendation:** Items that are most similar to the user profile are recommended to the user.

Content-based recommendation systems are less affected by the cold-start problem as they can still recommend items based on their features. They are also more interpretable as they rely on item attributes. However, they may miss out on providing serendipitous recommendations and can be limited by the quality of feature extraction and user profiles.

**Choosing Between Collaborative Filtering and Content-Based:**
Both collaborative filtering and content-based approaches have their strengths and weaknesses. The choice between them depends on the specific requirements of the recommendation system, the type of data available, and the user base. Hybrid approaches that combine collaborative filtering and content-based techniques are also common, aiming to leverage the strengths of both methods and mitigate their weaknesses.

In this mini-project, you'll be building both content based and collaborative filtering engines for the [MovieLens 25M dataset](https://grouplens.org/datasets/movielens/25m/). The MovieLens 25M dataset is one of the most widely used and popular datasets for building and evaluating recommendation systems. It is provided by the GroupLens Research project, which collects and studies datasets related to movie ratings and recommendations. The MovieLens 25M dataset contains movie ratings and other related information contributed by users of the MovieLens website.

**Dataset Details:**
- **Size:** The dataset contains approximately 25 million movie ratings.
- **Users:** It includes ratings from over 162,000 users.
- **Movies:** The dataset consists of ratings for more than 62,000 movies.
- **Ratings:** The ratings are provided on a scale of 1 to 5, where 1 is the lowest rating and 5 is the highest.
- **Timestamps:** Each rating is associated with a timestamp, indicating when the rating was given.

**Data Files:**
The dataset is usually split into three CSV files:

1. **movies.csv:** Contains information about movies, including the movie ID, title, genres, and release year.
   - Columns: movieId, title, genres

2. **ratings.csv:** Contains movie ratings provided by users, including the user ID, movie ID, rating, and timestamp.
   - Columns: userId, movieId, rating, timestamp

3. **tags.csv:** Contains user-generated tags for movies, including the user ID, movie ID, tag, and timestamp.
   - Columns: userId, movieId, tag, timestamp

First, import all the libraries you'll need.

In [222]:
import zipfile
import numpy as np
import pandas as pd
from urllib.request import urlretrieve
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

Next, download the relevant components of the MoveLens dataset. Note, these instructions are roughly based on the colab [here](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/recommendation-systems/recommendation-systems.ipynb?utm_source=ss-recommendation-systems&utm_campaign=colab-external&utm_medium=referral&utm_content=recommendation-systems#scrollTo=O3bcgduFo4s6).

In [223]:
print("Downloading movielens data...")

urlretrieve('http://files.grouplens.org/datasets/movielens/ml-100k.zip', 'movielens.zip')
zip_ref = zipfile.ZipFile('movielens.zip', 'r')
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


Before doing any kind of machine learning, it's always good to familiarize yourself with the datasets you'lll be working with.

Here are your tasks:

1. Spend some time familiarizing yourself with both the `movies` and `ratings` dataframes. How many unique user ids are present? How many unique movies are there?
2. Create a new dataframe that merges the `movies` and `ratings` tables on 'movie_id'. Only keep the 'user_id', 'title', 'rating' fields in this new dataframe.

In [224]:
# Spend some time familiarizing yourself with both the movies and ratings
# dataframes. How many unique user ids are present? How many unique movies
# are there?
# There are 943 unique user ids present in the ratings dataframe.
# There are 1682 unique movies present in the movies and ratings dataframe.
# There is also 1664 unique titles shown in the movies dataframe.
print(movies['movie_id'].nunique())
print(ratings['user_id'].nunique())
print(ratings['movie_id'].nunique())
print(movies['title'].nunique())

1682
943
1682
1664


In [225]:
# Merge movies and ratings dataframes
# Merge movies and ratings dataframes
# Using the merge function in pandas, I combined the movies and ratings together on the movie id column only keeping the 'user_id', 'title', 'rating' fields. Also by the default the merge specifies the inner joining method.
df=pd.merge(movies,ratings,on='movie_id')
df1 = df[['user_id', 'title', 'rating']]
df1

,user_id,title,rating
0,308,Toy Story (1995),4
1,287,Toy Story (1995),5
2,148,Toy Story (1995),4
3,280,Toy Story (1995),4
4,66,Toy Story (1995),3
...,...,...,...
99995,863,Mat' i syn (1997),1
99996,863,B. Monkey (1998),3
99997,863,Sliding Doors (1998),2
99998,896,You So Crazy (1994),3


As mentioned in the introduction, content-Based Filtering is a recommendation engine approach that focuses on the attributes or features of items (products, movies, music, articles, etc.) and leverages these features to make personalized recommendations. The underlying idea is to match the characteristics of items with the preferences of users to suggest items that align with their interests. Content-based filtering is particularly useful when explicit user-item interactions (e.g., ratings or purchases) are sparse or unavailable.

**Key Steps in Content-Based Filtering:**

1. **Feature Extraction:**
   - For each item, relevant features are extracted. These features are typically descriptive attributes that can be represented numerically, such as genre, director, actors, author, publication date, and keywords.
   - In the case of text-based items, natural language processing techniques may be used to extract features like TF-IDF (Term Frequency-Inverse Document Frequency) scores.

2. **User Profile Creation:**
   - A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.
   - For example, if a user has watched several action movies, the action genre feature would receive a higher weight in their profile.

3. **Similarity Calculation:**
   - The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity, Euclidean distance, or Pearson correlation.
   - Cosine similarity is commonly used as it measures the cosine of the angle between two vectors, which represents their similarity.

4. **Recommendation:**
   - Items that are most similar to the user profile are recommended to the user. These are items whose features have the highest similarity scores with the user profile.
   - The recommended items are presented as a list sorted by their similarity scores.

**Advantages of Content-Based Filtering:**
1. **No Cold-Start Problem:** Content-based filtering can make recommendations even for new users with no historical interactions because it relies on item features rather than user history.

2. **User Independence:** The recommendations are based solely on the features of items and do not require knowledge of other users' preferences or behavior.

3. **Transparency:** Content-based recommendations are interpretable, as they depend on the features of items, making it easier for users to understand why specific items are recommended.

4. **Serendipity:** Content-based filtering can recommend items with characteristics not seen before by the user, leading to serendipitous discoveries.

5. **Diversity in Recommendations:** The method can offer diverse recommendations since it suggests items with different feature combinations.

**Limitations of Content-Based Filtering:**
1. **Limited Discovery:** Content-based filtering may struggle to recommend items outside the scope of users' historical interactions or interests.

2. **Over-Specialization:** Users may receive recommendations that are too similar to their previous choices, leading to a lack of exposure to new item categories.

3. **Dependency on Feature Quality:** The quality and relevance of item features significantly influence the quality of recommendations.

4. **Limited for Cold Items:** Content-based filtering can struggle to recommend new items with limited feature information.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return content-based recommendations for this user. Here are steps you can take:

  A. Get the user's rated movies

  B. Create a TF-IDF matrix using movie genres. Note, this can be extracted from the `movies` dataframe.

  C. Compute the cosine similarity between movie genres. Use the [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) function.

  D. Get the indices of similar movies to those rated by the user based on cosine similarity. Keep only the top 5.

  E. Remove duplicates and movies already rated by the user.

In [248]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
# Content-Based Filtering using Movie Genres
def content_based_recommendation(user_id, df):
  # Get the user's rated movies
  # First the user rated movies user ids from the df dataframe are filtered where
  # the user_id from the input matches the value from the user_id column.
    users_rated_movies=df[df['user_id']==user_id]
  # From there we get each of the unique movie ids that were rated by that user.
  # from the movie_id's column using the unique function.
    rated_movie_ids = users_rated_movies['movie_id'].unique()
  # Create a TF-IDF matrix using movie genres
  # To collect each of the movie genres we only take indices 6 to -3 as they are
  # the ones that indicate the genres of the movies in those specific columns. Each
  # value in the column are binary with 1's indicating that fall under that genre,
  # and 0 being that do not.
  # Using the TfidfTransformer we can create a TF-IDF matrix to convert the binary genre data
  # to noralized TF-IDF scores, inorder to compute the similarities between the movies
  # based on their genres.
    tfidf = TfidfTransformer()
  # Using the fit transform we take the data of the genres columns using iloc to
  # convert it to a TF-IDF matrix.
    tfidf_matrix = tfidf.fit_transform(df.iloc[:,6:-3])
  # Get the indices of the similar movies based on cosine similarity
  # To start of with I want to create a empty set() where we can store our the similar
  # movie ids. I prefered using a set over a list to avoid using duplicate values.
    recommended_movies = set()
    # We go over each of the movie id that was rated by the user using a for loop.
    for movie_id in rated_movie_ids:
    # From there we obtain the index of the row for that movie_id. This is important
    # because we want to know which row in the TF-IDF matrix corresponds to this movie_id
    # inorder to compare it to other movie vectors in the TF-IDF matrix when computing the
    # cosine_similarity.
      movie_idx = df[df['movie_id'] == movie_id].index[0]
    # Using the cosine_similarity, we can compute that row from the tfidf_matrix
    # to the other movie vectors. We then use the flatten() function inorder to
    # get it into a 1D array of similarity scores.
      cos_similarity = cosine_similarity(tfidf_matrix[movie_idx], tfidf_matrix).flatten()
    # using the argsort() function on the similarity scores for cos_similarity, we
    # can obtain the indices of the movies sorted by the similarity scores in ascending order.
    # [::-1] is used to order it in descending order with the higher similarity being
    # displayed first, with [1:] being used to skip the first result which is
    # always the current movie itself.
      similar_movie_indices = cos_similarity.argsort()[::-1][1:]
    # We can now evaluate each individual similar movie one at a time using a
    # for loop on the indices of the similarities for each movie.
      for idx in similar_movie_indices:
      # Using the df.iloc[idx] we obtain the row from df that corresponds
      # to the index idx. From the row, we then obtain the value for the
      # movie_id giving us the unqiue identifier for the movie. This gives us
      # the movie that corresponds to the similarity index.
          similar_movie_id = df.iloc[idx]['movie_id']
      # We create an if statement to make sure that the reviewer does not rate
      # a movie that he as already rated. Stating the if similar_movie_id is not in
      # rated_movie_ids we then add the similar movie ids to our recommended_movies set.
          if similar_movie_id not in rated_movie_ids:
              recommended_movies.add(similar_movie_id)
      # From there, we stop after collecting the first 5 unique recommendations.
      # Using the break.
          if len(recommended_movies) >= 5:
              break

  # Remove duplicates and movies already rated by the user
  # From there I created the top 5 recommended_movies set to a list. Using
  # [:5] to get the first 5 values.
    top_5 = list(recommended_movies)[:5]
  # I then filtered out the top 5 unique movie titles from the dataframe df
  # and returned it as my function.
    top_5_movie_titles = df[df['movie_id'].isin(top_5)]['title'].unique()
    return top_5_movie_titles

The key idea behind collaborative filtering is that users who have agreed in the past will likely agree in the future. Instead of relying on item attributes or user profiles, collaborative filtering identifies patterns of user behavior and item preferences from the interactions present in the data.

**Types of Collaborative Filtering:**
There are two main types of collaborative filtering:

**Collaborative Filtering Process:**
The collaborative filtering process typically involves the following steps:

1. **Data Collection:**
   - Gather data on user-item interactions, such as movie ratings, product purchases, or article clicks.

2. **User-Item Matrix:**
   - Organize the data into a user-item matrix, where rows represent users, columns represent items, and the entries contain the users' interactions (e.g., ratings).

3. **Similarity Calculation:**
   - Calculate the similarity between users or items using similarity metrics such as cosine similarity, Pearson correlation, or Jaccard similarity.
   - For user-based collaborative filtering, user similarities are calculated, and for item-based collaborative filtering, item similarities are calculated.

4. **Neighborhood Selection:**
   - For each user or item, select the most similar users or items as the neighborhood.
   - The size of the neighborhood (the number of similar users or items to consider) is an important parameter to control the system's behavior.

5. **Prediction Generation:**
   - Predict the ratings for items that the target user has not yet interacted with by combining the ratings of neighboring users or items.

6. **Recommendation Generation:**
   - Recommend items with the highest predicted ratings to the target user.

**Advantages of Collaborative Filtering using User-Item Interactions:**
- Collaborative filtering is based solely on user interactions and does not require knowledge of item attributes, making it useful for cases where item data is sparse or unavailable.
- It can provide serendipitous recommendations, suggesting items that users may not have discovered on their own.
- Collaborative filtering can be applied in various domains, including e-commerce, music, movie, and content recommendations.

**Limitations of Collaborative Filtering:**
- The cold-start problem: Collaborative filtering struggles to recommend to new users or items with no or limited interaction history.
- It may suffer from sparsity when data is limited or when users have only interacted with a small subset of items.
- Scalability issues can arise with large datasets and an increasing number of users or items.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return collaborative filtering recommendations for this user based on a user-item interaction matrix. Here are steps you can take:

  A. Create the user-item matrix using Pandas' [pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

  B. Fill missing values with zeros in this matrix.

  C. Calculate user-user similarity matrix using cosine similarity.

  D. Get the array of similarity scores of the target user with all other users from the similarity matrix.

  E. Extract, say the the top 5 most similar users (excluding the target user).

  F. Generate movie recommendations based on the most similar users.

  G. Remove duplicate movies recommendations.

In [227]:
# Collaborative Filtering using User-Item Interactions
def collaborative_filtering_recommendation(user_id, df):
    # Create the user-item matrix
    # Using the pivot_table function to create a pivot table in pandas
    # I set my index='user_id', columns='movie_id', values='rating'. I set the
    # aggfunc= 'mean' just in case there are multiple ratings for a movie by
    # a user.
    user_item_matrix = df.pivot_table(index='user_id', columns='movie_id', values='rating', aggfunc='mean')
    # Fill missing values with 0 (indicating no rating)
    # Using the Fillna function in pandas we set all the null values to 0
    # for our user_item_matrix.
    user_item_matrix = user_item_matrix.fillna(0)
    # Calculate user-user similarity matrix using cosine similarity
    # Using the cosine similarity function we want to see how similar
    # the movie ratings are between each user.
    user_user_similarity_matrix = cosine_similarity(user_item_matrix)
    # Get the similarity scores of the target user with all other users
    # From the pivot table (user_item_matrix) we get the index of the row
    # corresponding to the giving user_id input.
    user_idx = user_item_matrix.index.get_loc(user_id)
    # We can then get the similarity scores for the user_idx with respect to
    # all other users by grabing the user index from the cosine_similarity matrix
    # showing the similarities between each user.
    similarity_scores = user_user_similarity_matrix[user_idx]
    # Find the top N most similar users (excluding the target user)
    # Using the argsort() function on the similarity scores for cos_similarity, we
    # can obtain the indices of the movies sorted by the similarity scores in ascending order.
    # [::-1] is used to order it in descending order with the higher similarity being
    # displayed first, with [1:6] being used to skip the first result which is
    # always the current movie itself and displaying the top 5 most similar users.
    similar_users = similarity_scores.argsort()[::-1][1:6]
    # We then can get the user_ids for the top 5 most similar users by taking the
    # index of similar_users from our pivot table (user_item_matrix).
    similar_user_ids = user_item_matrix.index[similar_users]
    # Generate movie recommendations based on the most similar users
    # To start of with I want to create a empty set() where we can store our the similar
    # movie ids. I prefered using a set over a list to avoid using duplicate values.
    recommended_movies = set()
    # We are looping over each of the 5 user ids that are the most similar.
    for idx in similar_user_ids:
    # From there we obtain the movie ratings of the similar user using their user id.
        similar_user_ratings = user_item_matrix.loc[idx]
    # We also obtain the user id of the target user inorder to get their ratings as well.
        target_user_ratings = user_item_matrix.loc[user_id]
    # A For loop is then created inorder to iterate over each movie id and rating
    # for the similar user. If the target user has not rated that certain movie
    # found in the similar users ratings, the movie id is then added into the recommended_movies
    # set. We know this because we filled in all the NaN to 0 to indicate no ratings.
        for movie_id, rating in similar_user_ratings.items():
            if target_user_ratings[movie_id] == 0:
                recommended_movies.add(movie_id)
    # Remove duplicates from recommendations
    # From there I created the top 5 recommended_movies set to a list and only
    # taking the first 5 values in the list [:5].
        top_5 = list(recommended_movies)[:5]
    # I then filtered out the top 5 unique movie titles from the dataframe df
    # and returned it as my function.
        top_5_movie_titles = df[df['movie_id'].isin(top_5)]['title'].unique()
        return top_5_movie_titles

Now, test your recommendations engines! Select a few user ids and generate recommendations using both functions you've written. Are the recommendations similar? Do the recommendations make sense?

In [241]:
import random
# From the random import I took a random seed
random.seed(42)
# I then created a list of all the unique user_ids.
x=list(df['user_id'].unique())
# Using random sample I took 5 random values from the list
user_ids = random.sample(x, 5)
# I will use these random values as inputs for the two recommendation functions.
user_ids

[754, 75, 250, 48, 79]

In [249]:
# Test the recommendation engines
# I created a for loop to go through each of the user_ids and print out the results
# for the content_based_recommendation of each user_id found in the user_ids list.
# the df is the merged movie and ratings dataframe and user_id one of the random
# values from the user_ids list within the for loop.
for user_id in user_ids:
    print(f'{user_id}')
    content_based_recommendations = content_based_recommendation(user_id, df)
    print(content_based_recommendations)


754
['Richard III (1995)' 'Raging Bull (1980)' 'Right Stuff, The (1983)'
 'George of the Jungle (1997)' 'Turbulence (1997)']
75
['Richard III (1995)' 'George of the Jungle (1997)'
 'Sense and Sensibility (1995)' 'Just Cause (1995)'
 'Land Before Time III: The Time of the Great Giving (1995) (V)']
250
['Reservoir Dogs (1992)' 'Sense and Sensibility (1995)'
 'Speechless (1994)' 'Just Cause (1995)'
 'Land Before Time III: The Time of the Great Giving (1995) (V)']
48
['Richard III (1995)' 'Sense and Sensibility (1995)'
 'Fire Down Below (1997)' 'Just Cause (1995)' 'Hostile Intentions (1994)']
79
['Crumb (1994)' 'Aristocats, The (1970)' 'Raging Bull (1980)'
 'Halloween: The Curse of Michael Myers (1995)'
 'Land Before Time III: The Time of the Great Giving (1995) (V)']


In [250]:
# Test the recommendation engines
# I created a for loop to go through each of the user_ids and print out the results
# for the collaborative_filtering_recommendation of each user_id found in the user_ids list.
# the df is the merged movie and ratings dataframe and user_id one of the random
# values from the user_ids list within the for loop.
for user_id in user_ids:
  print(f'{user_id}')
  collaborative_filtering_recommendations = collaborative_filtering_recommendation(user_id, df)
  print(collaborative_filtering_recommendations)

754
['Toy Story (1995)' 'GoldenEye (1995)' 'Four Rooms (1995)'
 'Get Shorty (1995)' 'Copycat (1995)']
75
['GoldenEye (1995)' 'Four Rooms (1995)' 'Get Shorty (1995)'
 'Copycat (1995)' 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)']
250
['Four Rooms (1995)' 'Get Shorty (1995)' 'Copycat (1995)'
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)' 'Babe (1995)']
48
['Toy Story (1995)' 'GoldenEye (1995)' 'Four Rooms (1995)'
 'Get Shorty (1995)' 'Copycat (1995)']
79
['GoldenEye (1995)' 'Four Rooms (1995)' 'Get Shorty (1995)'
 'Copycat (1995)' 'Babe (1995)']


In [251]:
# Are the recommendations similar?
# Based on the results I got from the random set samples, both recommendations
# dont seem to have much similarities. But I felt that was sort of expected as
# they approach the problem differently. Collaborative filtering would be based
# on more user behaviors or recommending items based on the preferences of similar users. Content based filtering
# on the other hand, relies more on movie features to recommend items that are similar to what the user has previously rated highly.


# Do the recommendations make sense?
# Yes, both approaches provide relevant recommendations, though they differ in the factors they consider.
# Even though the recommendations aren't closely aligned, both methods still offer valuable insights from different perspectives.